In [1]:
import cv2 as cv
import numpy as np
import os
import math


In [ ]:
 
def click_corners(img, objpoints, imgpoints, objp, window, n_rows=9, n_cols=6):
    """
    get 4 corners from user clicks (in order) and use linear interpolation to get the 
    other inner points.
    """

    order = ['TL', 'BL', 'TR', 'BR']
    box_corners = []
    
    # function to handle clicks on the image  
    def click_event(event, x, y, flags, param): 
        
        box_corners, img, window = param
        
        # check for left mouse clicks 
        if event == cv.EVENT_LBUTTONDOWN and len(box_corners) < 4: 
            
            box_corners.append((x, y))
    
            # display clicked points
            # on the image window 
            font = cv.FONT_HERSHEY_SIMPLEX 
            cv.putText(img, order[len(box_corners)-1], (x,y), font, 
                        1, (255, 0, 0), 2) 
            cv.imshow(window, img)
 
    cv.namedWindow(window, cv.WINDOW_NORMAL)
    cv.imshow(window, img) 
    cv.setMouseCallback(window, click_event, [box_corners, img, window]) 
    # wait for a key to be pressed to exit 
    print("click on the 4 corners, then press any key.")
    cv.waitKey(0) 

    tl, bl, tr, br = np.array(box_corners, dtype=np.float32)

    # get first and last columns of points with linear interpolation
    first_col = np.linspace(tl, bl, n_rows) 
    last_col = np.linspace(tr, br, n_rows) 

    # get rest corner points by linearly interpolating the two columns
    all_points = np.vstack([
        np.linspace(first_col[i], last_col[i], n_cols) for i in range(n_rows)
    ])
    corners = all_points.reshape(-1, 1, 2)

    objpoints.append(objp)
    imgpoints.append(corners)

    cv.destroyWindow(window)
    cv.drawChessboardCorners(img, (n_rows, n_cols), corners, True)
    cv.imshow("Interpolated Chessboard Corners", img)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [ ]:
def find_auto(img, gray, objpoints, imgpoints, objp, window, n_rows=9, n_cols=6):
    """
    used to find the corner points and fill objpoints, imgpoints lists
    img, gray: original and grayscale image
    """

    # choice task - denoising
    gray = cv.fastNlMeansDenoising(gray, None, h=10)

    # detect chess board corners
    ret, corners = cv.findChessboardCorners(gray, (n_rows,n_cols), None)

    if ret:        
        # improve quality of automatically found corners    
        criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
        corners = cv.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)

        objpoints.append(objp)
        imgpoints.append(corners)
        
        # display the corners on image
        cv.drawChessboardCorners(img, (n_rows,n_cols), corners, True)
        cv.imshow(window, img)
        cv.waitKey(0)
        cv.destroyWindow(window)
    else:
        # if corners are not found, switch to manual mode
        click_corners(img, objpoints, imgpoints, objp, window, n_rows, n_cols)


In [ ]:
def run(data='./data' , n_rows=9, n_cols=6, mode=1):
    """
    offline phase 
    mode defines the run setting (1,2,3)
    (n_rows, n_cols): the number of inner points to look for
    in our case and pattern we have (9,6)
    """

    images= os.listdir('./data') # mode 1 (using all images)

    if mode==2:
        images = images[-10:] 
    if mode==3:
        images = images[-5:] 
       
    objp = np.zeros((n_rows*n_cols,3), np.float32)
    objp[:,:2] = np.mgrid[0:n_rows,0:n_cols].T.reshape(-1,2)
    objp= objp*16.5  # we measured the side of each square to be 16,5mm  

    # Lists to store object and image points from all images.
    objpoints = [] # 3d point in real world space
    imgpoints = [] # 2d points in image plane.
    
    for fname in images:
        
        img = cv.imread(f'./data/{fname}')
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        # we convert to grayscale before passing to 'findChessboardCorners'

        window = f'image_{fname}'
        # find chess board corners either automatically or manually
        find_auto(img, gray, objpoints, imgpoints, objp, window, n_rows=n_rows, n_cols=n_cols)

    cv.destroyAllWindows()    
    # caliibrate camera and return results
    ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

    return ret, mtx, dist, rvecs, tvecs

In [5]:
def draw_cube(img, imgpts):
    
    imgpts = np.int32(imgpts).reshape(-1,2)

    # Define cube edge pairs (8 corners, connecting edges)
    edges = [
        (0,1), (1,2), (2,3), (3,0),  # Bottom face
        (4,5), (5,6), (6,7), (7,4),  # Top face
        (0,4), (1,5), (2,6), (3,7)   # Vertical edges
    ]

    # Draw cube edges
    for edge in edges:
        i, j = edge
        img = cv.line(img, tuple(imgpts[i]), tuple(imgpts[j]), (0, 255, 255), 3)


    return img

In [6]:
def fill_top_face_with_hsv(img, cube_3d_points, rvec, tvec, mtx, dist):
    """
    1) Identify the top face (corners #4..7).
    2) Compute color based on:
       - Distance from camera => Value
       - Angle of face normal => Saturation
       - Position => Hue
    3) Fill the top face in the image with that HSV-based color.
    """

    # In your cube definition, corners #4..7 form the "top" face (assuming +Z is 'up')
    top_face_idxs = [4,5,6,7]
    top_face_3d = cube_3d_points[top_face_idxs]  # shape (4,3)

    # 1) Project to 2D for filling in the image
    imgpts_top, _ = cv.projectPoints(top_face_3d, rvec, tvec, mtx, dist)
    imgpts_top = np.int32(imgpts_top).reshape(-1,2)  # 4 points (x,y)

    # 2) Compute the center of the top face in object coords
    center_3d_obj = np.mean(top_face_3d, axis=0)  # (x,y,z)

    # 3) Convert that center to camera coords => center_cam = R*center_3d + tvec
    R, _ = cv.Rodrigues(rvec)  # rotation matrix from rvec
    center_cam = R.dot(center_3d_obj) + tvec.ravel()  # shape (3,)

    # Distance from the camera to the center of the top face
    dist_to_cam = np.linalg.norm(center_cam)  # in same units as your calibration (mm, etc.)

    # 4) Face normal in object coords (since top face is parallel to X-Y, normal = +Z)
    #    If your top face is actually at negative Z, or you want the opposite direction,
    #    adjust accordingly.  We'll assume +Z is "up."
    normal_obj = np.array([0,0,1], dtype=np.float32)
    '''
    xedge1 = top_face_3d[1] - top_face_3d[0]
    edge2 = top_face_3d[3] - top_face_3d[0]
    normal_obj = np.cross(edge1, edge2)
    normal_obj = normal_obj / np.linalg.norm(normal_obj)  
    '''


    # normal_cam = R * normal_obj
    normal_cam = R.dot(normal_obj)

    # The camera's viewing direction is usually +Z in OpenCV. Let's define that:
    camera_axis = np.array([0,0,1], dtype=np.float32)

    # 5) Angle between face normal and camera axis
    #    angle = arccos( dot(n_cam, cam_axis)/(||n_cam||*||cam_axis||) ) in degrees
    dotval = np.dot(normal_cam, camera_axis)
    norms  = np.linalg.norm(normal_cam)*np.linalg.norm(camera_axis)
    # Guard numerical domain for arccos
    cos_angle = np.clip(dotval / (norms + 1e-9), -1.0, 1.0)
    angle_deg = math.degrees(math.acos(cos_angle))

    # 6) Compute HSV components
    #    V: scale linearly from 255 at dist=0, down to 0 at dist=4m (4000 mm for example).
    #    clamp if dist > 4m => V=0, if dist < 0 => V=255.
    max_dist = 4000.0  # 4 meters in mm (assuming your chessboard is in mm)
    V = 255.0*(1.0 - dist_to_cam/max_dist)
    V = max(0, min(255, V))

    #    S: scale from 255 at angle=0 to 0 at angle=45 or more
    max_angle = 45.0
    if angle_deg >= max_angle:
        S = 0
    else:
        S = 255.0*(1.0 - angle_deg/max_angle)
    S = max(0, min(255, S))

    #    H: pick any function you want that depends on position/orientation
    #    For example, vary hue based on the camera's X position of the center.
    #    We'll map X in camera coords from -500..+500 to 0..180. 
    #    That means if your center_cam[0] < -500 => hue=0, if > +500 => hue=180
    x_cam = center_cam[0]
    hue_min, hue_max = 0, 180
    x_min,  x_max    = -500, 500  # range in mm
    if x_cam <= x_min:
        H = hue_min
    elif x_cam >= x_max:
        H = hue_max
    else:
        # linear interpolation
        H = hue_min + (hue_max - hue_min) * (x_cam - x_min)/(x_max - x_min)

    # Convert HSV -> BGR for OpenCV fill
    # HSV is (H in [0..180], S in [0..255], V in [0..255]) in OpenCV's scale.
    # But if we want to call cv2.cvtColor, we need a 3D array of shape (1,1,3)
    # or use python's colorsys (which expects [0..1] range). We'll just do OpenCV style:

    hsv_color = np.uint8([[[H, S, V]]])  # shape (1,1,3)
    bgr_color = cv.cvtColor(hsv_color, cv.COLOR_HSV2BGR)[0][0]  # shape (3,)

    # 7) Fill the top face polygon
    #    We'll do fillConvexPoly so we get a nice solid face
    bgr_tuple = tuple(map(int, bgr_color))  # e.g. (R, G, B)
    cv.fillConvexPoly(img, imgpts_top, bgr_tuple)

    return img

In [7]:
def drawAxesCube(img, mtx, dist):
    
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    ret, corners = cv.findChessboardCorners(gray, (9,6), None)
    
    if not ret:
        return img

    criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    corners = cv.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)

    objp = np.zeros((9*6, 3), np.float32)
    objp[:,:2] = np.mgrid[0:9, 0:6].T.reshape(-1, 2)
    objp = objp * 16.5  # your real square size in mm

    _, rvec, tvec = cv.solvePnP(objp, corners, mtx, dist)

    square = 16.5
    cube_side = 2 * square  # cube side size =2 board squares
    
    # define axis
    axis = np.float32([[0,0,0],
                        [5*square,0,0],
                        [0,5*square,0],
                        [0,0,-5*square]])
                        
    # define cube
    cube = np.float32([
        [0,0,0],
        [cube_side, 0, 0],
        [cube_side, cube_side, 0],
        [0, cube_side, 0],
        [0, 0, -cube_side],
        [cube_side, 0, -cube_side],
        [cube_side, cube_side, -cube_side],
        [0, cube_side, -cube_side]
    ])

    imgpts_axis, _ = cv.projectPoints(axis, rvec, tvec, mtx, dist)
    imgpts_cube, _ = cv.projectPoints(cube, rvec, tvec, mtx, dist)

    origin = tuple(map(int, imgpts_axis[0].ravel()))
    pt_x   = tuple(map(int, imgpts_axis[1].ravel()))
    pt_y   = tuple(map(int, imgpts_axis[2].ravel()))
    pt_z   = tuple(map(int, imgpts_axis[3].ravel()))
    cv.circle(img, origin, 5, (0, 255, 255), -1)

    cv.arrowedLine(img, origin, pt_x, (0,0,255), 3, tipLength=0.1)  # X
    cv.arrowedLine(img, origin, pt_y, (0,255,0), 3, tipLength=0.1)  # Y
    cv.arrowedLine(img, origin, pt_z, (255,0,0), 3, tipLength=0.1)  # Z
    cv.putText(img, 'X', pt_x, cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv.LINE_AA)
    cv.putText(img, 'Y', pt_y, cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv.LINE_AA)
    cv.putText(img, 'Z', pt_z, cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv.LINE_AA)

    # draw cube on image
    img = draw_cube(img, imgpts_cube)
    img = fill_top_face_with_hsv(img, cube, rvec, tvec, mtx, dist)

    return img


In [ ]:
def online(setting='t', mtx=None, dist=None):
    """
    setting t -> perform online phase on test image
    setting c -> perform online phase on frames captured by webcam
    requires camera matrix and distortion coeffs from camera calibration
    """

    if setting=='c':
        # use webcam
        cam = cv.VideoCapture(0)

        while True:
            # get frame
            ret, frame = cam.read()
            
            if not ret:
                print("no frame")
                break
            
            # draw axes and cube if chessboard pattern is found 
            frame = drawAxesCube(frame, mtx=mtx, dist=dist)
            cv.imshow("Webcam", frame)
            
            # click q to exit and close webcam
            if cv.waitKey(1) & 0xFF == ord('q'):
                cam.release()
                cv.destroyAllWindows()
                break

    elif setting=='t':
        # read test image
        test_img = './test.jpg'
        img = cv.imread(test_img)
        # draw cube and axes using camera intrinsics
        image = drawAxesCube(img, mtx=mtx, dist=dist)
        cv.imshow("Webcam", image)
        cv.waitKey(0)
        cv.destroyAllWindows()


In [ ]:
for i in range(1,4):

    print(f"Run {i}")
    # offline phase
    ret, mtx, dist, rvecs, tvecs = run(n_rows=9, n_cols=6, mode=i)
    print(f"Ret {ret}, mtx {mtx}, dist {dist}, rvecs {rvecs}, tvecs {tvecs}")
    # online phase
    online(setting='t', mtx=mtx, dist=dist)

Run 1
click on the 4 corners, then press any key.
click on the 4 corners, then press any key.
click on the 4 corners, then press any key.
click on the 4 corners, then press any key.
Ret 17.31440125432973, mtx [[3.49303287e+03 0.00000000e+00 6.64566006e+02]
 [0.00000000e+00 6.64594378e+03 3.53932428e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]], dist [[-1.51293249e+00  4.40900003e+02  4.85676362e-02  4.44729808e-02
   6.04251830e+00]], rvecs (array([[ 0.45554426],
       [ 1.32856303],
       [-0.28329403]]), array([[-0.77473117],
       [-0.92241589],
       [-1.48445548]]), array([[ 1.36315313],
       [ 1.0033655 ],
       [-1.21745325]]), array([[ 0.46231067],
       [ 1.32627056],
       [-0.28908709]]), array([[-1.37930171],
       [-1.00737972],
       [-1.43694669]]), array([[-0.81895037],
       [-0.79157792],
       [-1.429086  ]]), array([[-0.77204763],
       [-0.86672867],
       [-1.54191103]]), array([[-0.7348298 ],
       [-0.91920828],
       [-1.60928692]]), ar

In [9]:
# Coice task 1 (real-time performance with webcam)
ret, mtx, dist, rvecs, tvecs = run(n_rows=9, n_cols=6, mode=3)
online(setting='c', mtx=mtx, dist=dist)
# press q to close window